## データセット（正弦波、矩形波、三角波）の作成

In [ ]:
# @markdown  学習データを作成します

# 学習データ作成

import numpy as np
from scipy import signal

np.random.seed(42)

DATA_NUM = 50
kinds = ["sine", "square", "triangle"]
fs = 8000
duration = 0.5
t = np.linspace(0, duration, int(fs*duration), endpoint=False)

def gen(freq, phase, amp, kind):
    if kind=="sine": return amp*np.sin(2*np.pi*freq*t+phase)
    if kind=="square": return amp*signal.square(2*np.pi*freq*t+phase)
    if kind=="triangle": return amp*signal.sawtooth(2*np.pi*freq*t+phase, 0.5)

X = []
y = []

for cls, kind in enumerate(kinds):
    for _ in range(DATA_NUM):
        freq  = 440*np.random.uniform(0.9, 1.1)
        phase = np.random.uniform(0, 2*np.pi)
        amp   = np.random.uniform(0.9, 1.1)

        wave = gen(freq, phase, amp, kind)
        X.append(wave)
        y.append(cls)

X = np.array(X)
y = np.array(y)

print(f'{DATA_NUM}個 × {len(kinds)}種類の波形 {kinds} をランダムに作成しました。')
print(f'周波数・位相・振幅をわずかにランダムに変えたデータになっています。')
print(f'サンプリング周波数が {fs}Hz、長さが {duration}秒 という設定のため、１波形あたり {int(fs*duration)} サンプルのデータがあります。')

In [ ]:
# @markdown 学習データを可視化します

import matplotlib.pyplot as plt

# =========================
# クラス別の件数
# =========================
counts = np.bincount(y)
k = len(counts)

x = np.arange(k)

plt.figure(figsize=(7, 3))
plt.bar(x, counts, label="All", color="#2ca02c")

plt.xticks(x, [f"class {i}" for i in range(k)])
plt.title("Class distribution (All data)")
plt.ylabel("Number of samples")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

print(f"全データとしては{DATA_NUM*k}個あり、各class {DATA_NUM}個ずつです。")
print("各データは1つの波形データから構成されています。")

In [ ]:
# @markdown 作成した波形からランダムに１つ選択し音を確認できます（実行のたびに変わります）

# 可視化：ランダムに選び音確認

VOLUME = 0.1

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

def show_sample(idx, play_audio=True):
    """
    X[idx] の波形を
    1) 全時間表示（audioなし）
    2) 先頭0.02秒を拡大表示
    し、必要なら音も再生する
    """
    wave = X[idx]
    label = y[idx]

    label_names = {i: name for i, name in enumerate(kinds)}
    name = label_names.get(label, f"class {label}")

    time_axis = np.arange(len(wave)) / fs

    print(f"index={idx}, class={label} ({name})")

    # -------------------------
    # 1) 全時間の波形（audioなし）
    # -------------------------
    plt.figure(figsize=(10, 2.6))
    plt.plot(time_axis, wave)
    plt.title(f"Full waveform")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    print(f"波形全体としては {duration}秒あります。")
    print("")

    # -------------------------
    # 2) 先頭0.02秒だけ拡大表示
    # -------------------------
    n_show = int(fs * 0.02)

    plt.figure(figsize=(10, 2.6))
    plt.plot(time_axis[:n_show], wave[:n_show])
    plt.title(f"Zoom (first 0.02s)")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    if play_audio:
        return Audio(wave * VOLUME, rate=fs, normalize=False)
    return None


# idx番目のサンプルを表示＆再生
idx = np.random.randint(0, len(X))
audio_obj = show_sample(idx, play_audio=True)

print("波形の形を確認しやすくするため、先頭0.02秒のみを表示しています。")
print("")

if audio_obj is not None:
    display(audio_obj)

print("コードを実行するたびに、ランダムに選ぶサンプルが変わります。")

## データセット分割



In [ ]:
# @markdown  学習データを分割します

# =========================
# データ分割（train / test）
# =========================
from sklearn.model_selection import train_test_split


print("データ:", X.shape)

num_classes = len(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,           # 20%をテストに
    random_state=42,
    stratify=y               # ここで各クラスの比率を保つ
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train class counts:", np.bincount(y_train))
print("Test  class counts:", np.bincount(y_test))
print("")
print("データが Train（訓練用）と Test（評価用）に分割されました")

In [ ]:
# @markdown 分割結果を可視化します


#可視化

# 色を統一（matplotlibデフォルト）
COLOR_TRAIN = "#2ca02c"  # green
COLOR_TEST  = "#ff7f0e"  # orange


# =========================
# クラス別の件数（比率維持の確認）
# =========================

train_counts = np.bincount(y_train)
test_counts  = np.bincount(y_test)

# クラス数を揃える
k = max(len(train_counts), len(test_counts))
train_counts = np.pad(train_counts, (0, k - len(train_counts)))
test_counts  = np.pad(test_counts,  (0, k - len(test_counts)))

x = np.arange(k)
w = 0.35

plt.figure(figsize=(7, 3))
plt.bar(x - w/2, train_counts, w,
        label="Train", color=COLOR_TRAIN)
plt.bar(x + w/2, test_counts,  w,
        label="Test",  color=COLOR_TEST)

plt.xticks(x, [f"class {i}" for i in range(k)])
plt.title("Class distribution (Train vs Test)")
plt.ylabel("Number of samples")
plt.grid(True, axis="y", alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


print(f"各classにおいて、データがTrain（訓練用）で{train_counts[0]}個、Test（評価用）で{test_counts[0]}個に分かれている様子です。")

## Echo State Network

### W_in, W

In [ ]:
# @markdown  W_inとWの作成

N_RESERVOIR = 20
SPECTRAL_RADIUS = 0.9
LEAK_RATE = 0.3
DENSITY = 0.1

class SimpleESN:
    def __init__(self, n_inputs=1, n_reservoir=20,
                 spectral_radius=0.9, input_scaling=0.8,
                 leak_rate=0.3, ridge_reg=1e-6,
                 density=0.1,
                 seed=0):

        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.spectral_radius = spectral_radius
        self.input_scaling = input_scaling
        self.leak_rate = leak_rate
        self.ridge_reg = ridge_reg
        self.density = density

        rng = np.random.default_rng(seed)

        # =========================
        # 入力＋バイアス → リザバー
        # =========================
        self.W_in = (rng.uniform(-1, 1, (n_reservoir, n_inputs)) * self.input_scaling)
        self.b_in = (rng.uniform(-1, 1, (n_reservoir,)) * self.input_scaling)

        # =========================
        # リザバー内部結合（density 対応）
        # =========================
        W = rng.uniform(-1, 1, (n_reservoir, n_reservoir))

        # 疎結合マスク
        mask = rng.uniform(0, 1, (n_reservoir, n_reservoir)) < density
        W = W * mask

        # =========================
        # スペクトル半径を調整
        # =========================
        eigvals = np.linalg.eigvals(W.astype(np.float64))
        sr = np.max(np.abs(eigvals))

        # sr=0 防止（density が極端に小さい場合）
        if sr == 0:
            raise ValueError("spectral radius is zero. Increase density.")

        self.W = W * (self.spectral_radius / sr)

        self.W_out = None  # 学習時に決まる

# W_inとWの作成
esn = SimpleESN(
    n_inputs=1,
    n_reservoir=N_RESERVOIR,
    spectral_radius=SPECTRAL_RADIUS,
    input_scaling=0.8,
    leak_rate=LEAK_RATE,
    ridge_reg=1e-5,
    density=DENSITY,
    seed=1234
)

W_in = esn.W_in  # (N_RESERVOIR, 1)
b_in = esn.b_in  # (N_RESERVOIR,)
W = esn.W  # (N_RESERVOIR, N_RESERVOIR)
n_res, n_col = W_in.shape

print("以下の設定値でWinとWを作成しました。")
print(f"・リザバーのノード数（N_RESERVOIR）: {N_RESERVOIR}")
print(f"・スペクトル半径（SPECTRAL_RADIUS）: {SPECTRAL_RADIUS}")
print(f"・リーク率（LEAK_RATE）: {LEAK_RATE}")
print(f"・結合密度（DENSITY）: {DENSITY}")

In [ ]:
# @markdown W_in と b_in を可視化します

import numpy as np
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(7, 4))

# --------------------
# W_in（入力重み）
# --------------------
im0 = axes[0].imshow(W_in, aspect="auto", interpolation="nearest")
axes[0].set_title(f"W_in : {n_res}×1")
axes[0].set_xticks([0])
axes[0].set_xticklabels(["input"])
axes[0].set_yticks(np.arange(0, n_res, max(1, n_res // 10)))
axes[0].set_ylabel("N_RESERVOIR")

plt.colorbar(im0, ax=axes[0], fraction=0.046)

# --------------------
# b_in（入力バイアス）
# --------------------
im1 = axes[1].imshow(b_in.reshape(-1, 1), aspect="auto", interpolation="nearest")
axes[1].set_title(f"b_in : {n_res}")
axes[1].set_xticks([0])
axes[1].set_xticklabels(["bias"])
axes[1].set_yticks([])  # 左で十分なので省略

plt.colorbar(im1, ax=axes[1], fraction=0.046)

plt.tight_layout()
plt.show()

print(f"W_inとb_inはいずれもリザバー数 {n_res} に対応しており、各要素は -1.0～+1.0 の一様乱数で初期化されています。")
print("バイアス（bias）を加えることで、入力が0付近でもリザバーが動き続け、時間方向の情報が保持されやすくなります。")

In [ ]:
# @markdown Wを可視化します

# Wの可視化

fig, ax = plt.subplots(figsize=(4, 4))
im = ax.imshow(W, aspect="auto")
fig.colorbar(im, ax=ax, label="weight")

# 軸ラベル（見やすく間引き）
ax.set_xticks(np.arange(0, n_res, 40))
ax.set_yticks(np.arange(0, n_res, 40))
ax.set_xticklabels([str(i) for i in range(0, n_res, 40)])
ax.set_yticklabels([str(i) for i in range(0, n_res, 40)])

ax.set_title(f"W : {n_res}×{n_res}") # (rows=reservoir, cols=reservoir)
ax.set_xlabel(f"N_RESERVOIR ({n_res})")
ax.set_ylabel(f"N_RESERVOIR ({n_res})")

plt.tight_layout()
plt.show()

print(f"Wが{n_res}×{n_res}の行列で、各要素の多くがゼロであることが分かります。これは、結合密度として {DENSITY} を設定しているためです。")
print(f"また、W はスペクトル半径を基準にスケーリングされており、その結果として各要素の最大値はおよそ ±{np.max(np.abs(W)):.2f} になっています。")

### u_seq, states

In [ ]:
# @markdown  u_seqを入力とし、W_inとWを使って各時刻のリザバー状態statesを作成

def _forward_states(self, u_seq):
    """
    1サンプル分の時系列 u_seq (T,) を受け取り、
    各時刻のリザバー状態 x(t) を返す: (T, n_reservoir)
    """
    T = len(u_seq)
    x = np.zeros(self.n_reservoir)
    states = np.zeros((T, self.n_reservoir), dtype=np.float32)

    for t in range(T):
        u = u_seq[t]
        # 入力＋バイアス
        u_vec = np.array([u], dtype=np.float32) # n_inputs=1想定
        pre_activation = (self.W_in @ u_vec) + self.b_in + (self.W @ x)
        x_new = np.tanh(pre_activation)
        # リーク付き更新
        x = (1 - self.leak_rate) * x + self.leak_rate * x_new
        states[t] = x

    return states

SimpleESN._forward_states = _forward_states

print("u_seq は訓練用学習データのうちの1サンプルです。")
print("u_seqをリザバーに入力し、各時刻のリザバー状態statesを作成しました。")

In [ ]:
# @markdown ランダムに波形を１つ選び、u_seqをとstatesを可視化（実行のたびに変わります）

# 可視化（先頭と末尾を同じ長さで表示）

i = np.random.randint(len(X_train))  # ランダムに1サンプル選択

# ラベル名
label = y_train[i]
label_names = {i: name for i, name in enumerate(kinds)}
name = label_names.get(label, f"class {label}")
print(f"index={i}, class={label} ({name})")

u_seq  = X_train[i]
states = esn._forward_states(u_seq)

T = len(u_seq)

# ====== ここだけ変えればOK ======
SHOW_LEN   = 300        # 左右共通：表示する時系列長
NODES_SHOW = [0, 1, 2]  # 表示するノード
# ==============================

# 範囲（右は末尾から自動）
left_start, left_end = 0, min(SHOW_LEN, T)
right_end   = T
right_start = max(0, T - SHOW_LEN)

fig = plt.figure(figsize=(11, 4))
gs = fig.add_gridspec(2, 3, width_ratios=[1, 0.05, 1])

# ========== 左：先頭 ==========
ax_u_l = fig.add_subplot(gs[0, 0])
ax_s_l = fig.add_subplot(gs[1, 0])

ax_u_l.plot(range(left_start, left_end), u_seq[left_start:left_end], color="black")
ax_u_l.set_title(f"u_seq: {left_start}–{left_end-1}")
ax_u_l.set_ylabel("u(t)")

for n in NODES_SHOW:
    ax_s_l.plot(range(left_start, left_end), states[left_start:left_end, n], label=f"node {n}")
ax_s_l.set_title(f"states: {left_start}–{left_end-1}")
ax_s_l.set_ylabel("x(t)")
ax_s_l.set_xlabel("t")
ax_s_l.legend()

# ========== 中央：… ==========
ax_mid_top = fig.add_subplot(gs[0, 1])
ax_mid_bot = fig.add_subplot(gs[1, 1])
for ax in [ax_mid_top, ax_mid_bot]:
    ax.axis("off")
    ax.text(0.5, 0.5, "...", ha="center", va="center", fontsize=24)

# ========== 右：末尾 ==========
ax_u_r = fig.add_subplot(gs[0, 2])
ax_s_r = fig.add_subplot(gs[1, 2])

ax_u_r.plot(range(right_start, right_end), u_seq[right_start:right_end], color="black")
ax_u_r.set_title(f"u_seq: {right_start}–{right_end-1}")

for n in NODES_SHOW:
    ax_s_r.plot(range(right_start, right_end), states[right_start:right_end, n])
ax_s_r.set_title(f"states: {right_start}–{right_end-1}")
ax_s_r.set_xlabel("t")

plt.tight_layout()
plt.show()

print("・上段が入力 u_seq 、下段がリザバー状態 states です。")
print("・u_seq は X_train の 1サンプルで、states はその入力に対する各時刻のリザバー状態です。")
print(f"・表示ノードは {NODES_SHOW} です（全ノード数: {states.shape[1]}）。")
print(f"・全時刻は 0–{T-1} ですが、長いので先頭 {left_end-left_start} 点と末尾 {right_end-right_start} 点のみ表示しています。")


### W_out

In [ ]:
# @markdown  W_outを学習して作成

def fit(self, X_seq, y_labels, num_classes):
    """
    X_seq: (N, T) の時系列データ（固定長）
    y_labels: (N,) のラベル (0,1,2)
    ※各フレームに「その発話のラベル」を付けて学習する
    """
    N, T = X_seq.shape

    # ---- 全フレームを縦に積む（N*T 行）----
    total_frames = N * T

    # R: (N*T, n_res)
    R = np.zeros((total_frames, self.n_reservoir), dtype=np.float32)

    # Y: (N*T, num_classes)
    Y = np.zeros((total_frames, num_classes), dtype=np.float32)

    row = 0
    for i in range(N):
        states = self._forward_states(X_seq[i])   # (T, n_res)
        R[row:row+T] = states

        # その発話ラベルを全フレームに付与
        Y[row:row+T, y_labels[i]] = 1.0
        row += T

    # ---- リッジ回帰 ----
    Rt = R.T
    I = np.eye(self.n_reservoir, dtype=np.float32)
    A = Rt @ R + self.ridge_reg * I
    B = Rt @ Y
    self.W_out = np.linalg.solve(
        A.astype(np.float64),
        B.astype(np.float64)
    ).astype(np.float32)

SimpleESN.fit = fit


esn.fit(X_train, y_train, num_classes)

print(f"学習が完了しました。")

In [ ]:
# @markdown 学習したW_outを可視化

# W_out の可視化（class を離散ラベル化）

fig, ax = plt.subplots(figsize=(5, 4))
im = ax.imshow(esn.W_out, aspect="auto", interpolation="nearest", cmap="coolwarm")
fig.colorbar(im, ax=ax, label="weight")

# ===== x軸：class ラベル（自動対応） =====
ax.set_xticks(np.arange(num_classes))
ax.set_xticklabels([f"class{i}" for i in range(num_classes)])

# ===== y軸：reservoir neuron（間引き） =====
ax.set_yticks(np.arange(0, n_res, 20))
ax.set_yticklabels([str(i) for i in range(0, n_res, 20)])

# ===== タイトル・ラベル =====
ax.set_title(f"W_out : {n_res}×{num_classes}")
ax.set_ylabel(f"N_RESERVOIR ({n_res})")

plt.tight_layout()
plt.show()

print(f"W_outが{n_res}×{num_classes}の行列であることが分かります。")
print("W_out は高次元に広がったリザバー状態をまとめた結果なので、人間の視覚では直感的に理解しにくい分布になっています。")

### Y

In [ ]:
# @markdown  評価用データを使って推論スコアを出力

def predict_proba(self, X_seq):
    """
    X_seq: (N, T)
    戻り値: (N, num_classes) （線形出力）
    ※各波形について「全フレームの線形出力を平均して」返す
    """
    N, T = X_seq.shape
    Y_lin = np.zeros((N, self.W_out.shape[1]), dtype=np.float32)  # (N, num_classes)

    for i in range(N):
        states = self._forward_states(X_seq[i])   # (T, n_reservoir)

        # 各フレームの線形出力: (T, num_classes)
        Z = states @ self.W_out

        # 発話内で集約（平均）
        Y_lin[i] = Z.mean(axis=0)

    return Y_lin

SimpleESN.predict_proba = predict_proba

# 推論（N, num_classes）
Y_lin = esn.predict_proba(X_test)

print(f"評価用データ X_test の {len(X_test)}個のデータを入れて、各クラスに対する出力 Y（linear score）を得ます。")

In [ ]:
# @markdown 推論スコア結果を可視化

# 可視化

plt.figure(figsize=(4, 4))
im = plt.imshow(Y_lin, aspect="auto", cmap="coolwarm")

plt.colorbar(im, label="linear score")
plt.ylabel("X_test")

# 軸ラベルを明示
plt.xticks(range(Y_lin.shape[1]), [f"class{i}" for i in range(Y_lin.shape[1])])
plt.yticks(range(len(Y_lin)), [f"{i}" for i in range(len(Y_lin))])

plt.title(f"Y : {len(Y_lin)}×{Y_lin.shape[1]}")
plt.tight_layout()
plt.show()

print(f"Y は {Y_lin.shape[0]}×{Y_lin.shape[1]} の行列で、推論するサンプル数 × クラス数を表しています。")
print(f"Y は、各 X_test サンプルについて、どのクラスに近いかをモデルが点数（linear score）として出力した結果です。")

### Predict

In [ ]:
# @markdown  推論スコアから推論クラスを決定

def predict(self, X_seq):
    """
    クラスID (argmax) を返す
    """
    Y_lin = self.predict_proba(X_seq)
    return np.argmax(Y_lin, axis=1)

SimpleESN.predict = predict

# 予測
y_pred = esn.predict(X_test)

print("（linear score）をもとに、各 X_test で最もスコアの高いクラスを予測します。")

In [ ]:
# @markdown 推論クラス結果を可視化

# argmax位置だけ 1、それ以外 0 の行列を作る
argmax_map = np.zeros_like(Y_lin)
for i, c in enumerate(y_pred):
    argmax_map[i, c] = 1

plt.figure(figsize=(4, 4))
im = plt.imshow(argmax_map, aspect="auto", cmap="Greens")

# plt.colorbar(im, label="selected (1 = chosen)")
plt.ylabel("X_test")

# 軸ラベルを明示（Y_lin と同じ）
plt.xticks(range(Y_lin.shape[1]), [f"class{i}" for i in range(Y_lin.shape[1])])
plt.yticks(range(len(Y_lin)), [f"{i}" for i in range(len(Y_lin))])

plt.title(f"Y (argmax) : {Y_lin.shape[0]}×{Y_lin.shape[1]}")
plt.tight_layout()
plt.show()

print(f"Y (argmax) は {Y_lin.shape[0]}×{Y_lin.shape[1]} の行列で、各 X_test サンプルについて、最もスコアが高かったクラスだけを 1 として示しています。")

In [ ]:
# @markdown 正解ラベル（y_test）を可視化します

num_classes = Y_lin.shape[1]              # class数はY_linに合わせる
N = len(y_test)

# 正解位置だけ1、それ以外0（one-hotマップ）
gt_map = np.zeros((N, num_classes), dtype=np.int32)
for i, c in enumerate(y_test):
    gt_map[i, c] = 1

plt.figure(figsize=(4, 4))
im = plt.imshow(gt_map, aspect="auto", cmap="Greens")

# plt.colorbar(im, label="correct (1 = true)")
plt.ylabel("X_test")

# 軸ラベル
plt.xticks(range(num_classes), [f"class{i}" for i in range(num_classes)])
plt.yticks(range(N), [f"{i}" for i in range(N)])

plt.title(f"y_test : {N}×{num_classes}")
plt.tight_layout()
plt.show()

print(f"y_test は {N}×{num_classes} の行列（one-hot）として可視化しており、各 X_test サンプルの正解クラスだけを 1 として示しています。")
print("先ほどの Y (argmax) の図と見比べて、一致しているところは推論が正しいことを意味します。")

## 性能評価

### 混同行列（Confusion Matrix）

In [ ]:
# @markdown 混同行列を求めます
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
# @markdown 混同行列を可視化します

num_classes = cm.shape[0]

plt.figure(figsize=(4, 4))
im = plt.imshow(cm, cmap="Blues")

plt.colorbar(im, label="count", shrink=0.8)

# 軸ラベル
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.xticks(range(num_classes), [f"class{i}" for i in range(num_classes)])
plt.yticks(range(num_classes), [f"class{i}" for i in range(num_classes)])

# 各セルに数値を表示（重要）
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(
            j, i, cm[i, j],
            ha="center", va="center",
            color="white" if cm[i, j] > cm.max() * 0.5 else "black"
        )

plt.title("Confusion Matrix (Test)")
plt.tight_layout()
plt.show()

print("混同行列は、推論結果と正解ラベルを同時に見て、どこで間違えているかを確認するための可視化です。")
print("対角線は正解した数、それ以外は間違えた数を示しています。")
print("この表を見ることで、どのクラスを混同しているかを確認できます。")

### 正解率（Accuracy）

In [ ]:

# @markdown  モデルに対し、TrainデータとTestデータそれぞれの正解率を計算します

from sklearn.metrics import accuracy_score

y_train_pred = esn.predict(X_train)
y_test_pred = y_pred #既に実施済

print("Train accuracy:", accuracy_score(y_train, y_train_pred))
print("Test  accuracy:", accuracy_score(y_test,  y_test_pred))

In [ ]:
# @markdown 正解率を可視化します

acc_train = accuracy_score(y_train, y_train_pred)
acc_test  = accuracy_score(y_test, y_test_pred)

plt.figure(figsize=(3, 3))
plt.bar(["Train", "Test"], [acc_train, acc_test])
plt.ylim(0, 1)
plt.ylabel("accuracy")
plt.title("Accuracy comparison")
plt.show()

print("Train と Test の精度を比べると、いくつか典型的なパターンがあります。")
print("・Train と Test ともに高い場合は、うまく学習できており汎化性能も良好です。")
print("・Train が高く Test が低い場合は、学習データに特化しすぎた過学習の可能性があります。")
print("・Train と Test ともに低い場合は、モデルの表現力や設定が不足していると考えられます。")

## 未知データでの推論

In [ ]:
# @markdown 新規データを１つ作成し可視化します（実行のたびに変わります）

# 可視化：新規生成データを1つ作って音確認

VOLUME = 0.1

def show_generated_sample():
    """
    gen() を使って新規データを1つ生成し、
    波形を表示して音を再生する
    """
    # kind をランダムに選択
    kind  = np.random.choice(kinds)
    freq  = 440 * np.random.uniform(0.9, 1.1)
    phase = np.random.uniform(0, 2*np.pi)
    amp   = np.random.uniform(0.9, 1.1)

    wave = gen(freq, phase, amp, kind)
    cls  = kinds.index(kind)

    # 波形プロット
    plt.figure(figsize=(10, 3))

    # 先頭0.02秒だけ拡大表示
    n_show = int(fs * 0.02)
    time_axis = np.arange(len(wave)) / fs

    plt.plot(time_axis[:n_show], wave[:n_show])
    plt.title(f"X_new, class={cls} ({kind})")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

    # 音を再生
    return wave, cls, Audio(wave * VOLUME, rate=fs, normalize=False)


# 新規生成サンプルを表示＆再生
wave_new, cls_new, audio_obj = show_generated_sample()
print(f"グラフでは先頭0.02秒の波形のみ表示していますが、データとしては{duration}秒あります。")
print("")

display(audio_obj)
print("コードを実行する度にランダムに波形が変わります。")

In [ ]:
# @markdown 推論して可視化します

# ---------- 推論 ----------
X_one = wave_new[None, :]                 # (1, T)
Y_lin_one = esn.predict_proba(X_one)      # (1, num_classes)
y_pred_one = esn.predict(X_one)           # (1,)

print(f"predicted class = {int(y_pred_one[0])} ({kinds[int(y_pred_one[0])]})")
print("")
print(kinds)

# ---------- 可視化：Y_lin（Y軸1つ・コンパクト） ----------
fig, ax = plt.subplots(figsize=(4, 2))

im = ax.imshow(Y_lin_one, aspect="equal", cmap="coolwarm")

cbar = fig.colorbar(im, ax=ax, label="linear score", shrink=0.7)

ax.set_ylabel("X_new")
ax.set_xticks(range(Y_lin_one.shape[1]))
ax.set_xticklabels([f"class{i}" for i in range(Y_lin_one.shape[1])])
ax.set_yticks([0])
ax.set_yticklabels(["0"])

ax.set_title(f"Y : {Y_lin_one.shape[0]}×{Y_lin_one.shape[1]}")
plt.tight_layout()
plt.show()

print("Y は、新規入力 X_new に対して、各クラスがどれだけ近いかを点数（linear score）で表した結果です。")
print("")



# ---------- 可視化：Y (argmax) ----------
argmax_map_one = np.zeros_like(Y_lin_one, dtype=np.int32)
argmax_map_one[0, int(y_pred_one[0])] = 1

fig, ax = plt.subplots(figsize=(4, 2))
im = ax.imshow(argmax_map_one, aspect="equal", cmap="Greens")
fig.colorbar(im, ax=ax, label="selected", shrink=0.7)

ax.set_ylabel("X_new")
ax.set_xticks(range(num_classes))
ax.set_xticklabels([f"class{i}" for i in range(num_classes)])
ax.set_yticks([0])
ax.set_yticklabels(["0"])

ax.set_title(f"Y (argmax) : {argmax_map_one.shape[0]}×{argmax_map_one.shape[1]}")
plt.tight_layout()
plt.show()


print("Y (argmax) は、その中で最もスコアが高かったクラスだけを 1 として示したものです。")